In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import analysis

This notebook compares sensitivities of individual variables, changing one-at-a-time (OAT). The rest of the variables remain as per baseline.

A positive `delta` indicates that buying would result in a financial gain over renting after 5 years.

In [ ]:
vars = [
    ("initialPropertyValue", 2),
    ("propertyValueYearlyIncrease", 1),
    ("mortgageInterestRate", 1),
    ("initialMonthlyRentPrice", 3),
    ("rentPriceYearlyIncrease", 1),
    ("savingsInterestRate", 1),
]


def scatter(var: str, slope: bool = False) -> go.Scatter:
    df = analysis.load_csv(var)
    y = df["delta"] if not slope else np.gradient(df["delta"])
    units = analysis.infer_units(var)
    units_prefix = units if units == "£" else ""
    units_suffix = units if units == "%" else ""
    return go.Scatter(
        x=df[var] * analysis.infer_multiplier_from_units(units),
        y=y,
        name=analysis.get_axis_title(var),
        mode="markers+lines",
        hovertemplate=f"{units_prefix}%{{x}}{units_suffix}, £%{{y}}",
    )

def print_slope_range(var: str, rate: float) -> None:
    df = analysis.load_csv(var)
    x_diff = df[var].diff()[1:]
    assert np.allclose(x_diff, rate), f"Expected uniform x difference of {rate}, got {x_diff.unique()}"
    y_diff = df["delta"].diff()
    print(f"For each {rate} increase in {var}, delta changes [£{y_diff.min():,.0f}, £{y_diff.max():,.0f}]")

In [ ]:
baseline = analysis.load_csv("baseline")["delta"].iloc[0]
print(f"Baseline delta: £{baseline:,.2f}")
fig = make_subplots(cols=3, horizontal_spacing=0.01)
for name, col in vars:
    fig.add_trace(scatter(name), row=1, col=col)
fig.add_hline(
    baseline,
    line_color="green",
    line_dash="dash",
    line_width=0.5,
    annotation_text="Baseline",
)
fig.add_hline(0, line_color="black", line_width=2)
y_range = [-135_000, 310_000]
fig.update_layout(
    xaxis1={"title_text": "Value (%)"},
    xaxis2={"title_text": analysis.get_axis_title("initialPropertyValue")},
    xaxis3={"title_text": analysis.get_axis_title("initialMonthlyRentPrice")},
    yaxis1={"range": y_range, "title_text": "Delta (£)"},
    yaxis2={"range": y_range, "showticklabels": False},
    yaxis3={"range": y_range, "showticklabels": False},
    legend={"xanchor": "center", "x": 0.5, "y": -0.2, "orientation": "h"},
)
fig.show()
analysis.save(fig, "1d-sensitivity")
print_slope_range("propertyValueYearlyIncrease", 0.01)
print_slope_range("mortgageInterestRate", 0.01)
print_slope_range("initialMonthlyRentPrice", 100)

In [ ]:
fig = make_subplots(cols=3, horizontal_spacing=0.01)
for name, col in vars:
    fig.add_trace(scatter(name, slope=True), row=1, col=col)
y_range = [-15_000, 25_000]
fig.update_layout(
    xaxis1={"title_text": "Value (%)"},
    xaxis2={"title_text": analysis.get_axis_title("initialPropertyValue")},
    xaxis3={"title_text": analysis.get_axis_title("initialMonthlyRentPrice")},
    yaxis1={"range": y_range, "title_text": "Slope"},
    yaxis2={"range": y_range, "showticklabels": False},
    yaxis3={"range": y_range, "showticklabels": False},
    legend={"xanchor": "center", "x": 0.5, "y": -0.2, "orientation": "h"},
)
fig.show()
analysis.save(fig, "1d-sensitivity-slope")